## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [9]:
import json
import numpy as np
from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel
from tqdm.auto import tqdm

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import ScoringReviewer
from lattereview.workflows import ReviewWorkflow

## Data

Building five example stories and dummy question-answering pairs from each story:

In [3]:
class BuildStoryOutput(BaseModel):
    story: str
    questions: list[str]
    answers: list[bool]

async def build_story():
    prompt = """
    Write a one-paragraph story with whatever realistic or imaginary theme you like,  
    then create three TRUE/FALSE questions based on your story. 
    Ensure that only readers of your story can determine whether the statements are true or false. 
    Do not reveal the answers to your questions.
    Return your story, a Python list of three questions, and another Python list of three boolean responses to the questions as your output.
    """
    provider = OpenAIProvider(model="gpt-4o", response_format_class=BuildStoryOutput)
    return await provider.get_json_response(prompt, temperature=0.9)

def run_build_story():
    response =  asyncio.run(build_story())[0]
    return response

data = {
    "question": [],
    "answer": [],
    "story": []
}
for i in tqdm(range(5)):
    out = json.loads(run_build_story())
    for j in range(3):
        data["question"].append(out["questions"][j])
        data["answer"].append(out["answers"][j])
        data["story"].append(out["story"])


data = pd.DataFrame(data)
data.to_csv("data.csv", index=False)
data

100%|██████████| 5/5 [00:15<00:00,  3.04s/it]


,question,answer,story
0,Aria lived in a city and had a pet cat named O...,False,"Once upon a time, in a small, enchanted villag..."
1,Aria discovered a hidden waterfall deep in the...,True,"Once upon a time, in a small, enchanted villag..."
2,The dragon Aria saved was as large as a horse.,False,"Once upon a time, in a small, enchanted villag..."
3,The village of Mistralwood is located near a d...,False,"In the quiet village of Mistralwood, nestled b..."
4,The lake is called Moonlit Mirror because it r...,True,"In the quiet village of Mistralwood, nestled b..."
5,Elara saw her own reflection and nothing else ...,False,"In the quiet village of Mistralwood, nestled b..."
6,Did Luna discover the grove by following a pat...,False,"In the heart of the Enchanted Forest, a tiny f..."
7,Were the flowers in the grove capable of speak...,True,"In the heart of the Enchanted Forest, a tiny f..."
8,Did Luna solve all three riddles immediately u...,False,"In the heart of the Enchanted Forest, a tiny f..."
9,Elara lived in a bustling city surrounded by s...,False,"In the small village of Windermere, nestled am..."


Embedding the stories to build a vector base:

In [4]:
data = pd.read_csv("data.csv")

async def get_embedding(text):
    client = AsyncOpenAI()
    if isinstance(text, str):
        text = [text]
    text = [x.replace("\n", " ") for x in text]         
    out = await client.embeddings.create(
        model="text-embedding-ada-002",
        input=text,
        encoding_format="float"
    )
    out = [np.array(x.embedding) for x in out.data]
    return out if len(out) > 1 else out[0]

stories = {story: None for story in set(data["story"].tolist())}

# Create async tasks for all embeddings
async def process_embeddings():
    tasks = [get_embedding(story) for story in stories.keys()]
    embeddings = await asyncio.gather(*tasks)
    return list(zip(embeddings, stories.keys()))

# Run the async code and get results
vector_story_pairs = await process_embeddings()
vector_base = np.array([x[0] for x in vector_story_pairs])
vector_base

array([[ 1.73057950e-02, -1.04124740e-02, -1.30683140e-02, ...,
         4.69549840e-03, -1.86238290e-02, -3.11583400e-02],
       [ 1.99574530e-02, -1.46224220e-02, -2.18749340e-02, ...,
        -5.61548000e-03, -1.93835140e-02, -1.72703710e-02],
       [ 1.82237530e-02,  2.50000950e-03, -1.87237560e-02, ...,
         4.71712300e-03,  1.11924760e-03, -3.96843630e-02],
       [ 7.52422700e-03, -2.52429920e-02, -1.28845805e-02, ...,
         4.84076140e-03, -1.67979670e-02, -2.20070480e-02],
       [ 1.90936740e-02,  6.87293000e-03, -5.26198400e-03, ...,
        -1.51455350e-02, -2.16474770e-06, -2.98421180e-02]])

## Retrieval

In [5]:
async def find_relevant_story(statement):
    s_embeddings = await get_embedding(statement)
    dot_product = np.dot(vector_base, s_embeddings)
    base_norms = np.linalg.norm(vector_base, axis=1)
    query_norm = np.linalg.norm(s_embeddings)
    cosine_similarities = dot_product / (base_norms * query_norm)
    retrieved_index = np.argmax(cosine_similarities)
    retrieved_story = vector_story_pairs[retrieved_index][1]
    return retrieved_story

input_index = 11
statement = data.iloc[input_index]["question"]
retrieved_story = await find_relevant_story(statement)

print(f"=== The question was chosen from row {input_index} ===\n{statement}")
print(f"=== The related story to the question ===\n{data.iloc[input_index]['story']}")
print(f"=== The retrieved Story ===\n{retrieved_story}")

=== The question was chosen from row 11 ===
The treasure was believed to be buried beneath the village square.
=== The related story to the question ===
In the small village of Windermere, nestled among the emerald hills of the countryside, lived a peculiar girl named Elara. She possessed a rare gift; she could communicate with the birds, who often brought her tales of distant lands and whispered secrets of the world. One brisk autumn morning, as the leaves fell gently around her, a majestic raven landed on her windowsill with a message from the kingdom across the sea. It spoke of a hidden treasure buried beneath the old oak tree at the edge of the village, a treasure that could bring prosperity to Windermere. Elara, with her adventurous spirit and loyal companion, a red squirrel named Sprocket, set out to uncover the truth behind the raven's tale.
=== The retrieved Story ===
In the small village of Windermere, nestled among the emerald hills of the countryside, lived a peculiar girl n

## Scoring with Retrieval Augmented Generation

In [6]:
reviewer = ScoringReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="reviewer",
    max_concurrent_requests=20, 
    backstory="A frequent book reader",
    input_description="TRUE/FALSE questions about stories",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Decide if the input statement is True or False given the provided story in the provided context",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the statement is TRUE and 2 if the statement is FALSE.',
    additional_context = find_relevant_story
)

review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [reviewer],
            "text_inputs": ["question"]
        }
    ]
)

updated_data = asyncio.run(review(data))
updated_data


====== Starting review round A (1/1) ======

Processing 15 eligible rows


['round: A', 'reviewer_name: reviewer'] -                     2024-12-24 21:56:10: 100%|██████████| 15/15 [00:02<00:00,  6.90it/s]

The following columns are present in the dataframe at the end of reviewer's reivew in round A: ['question', 'answer', 'story', 'round-A_reviewer_output', 'round-A_reviewer_reasoning', 'round-A_reviewer_score', 'round-A_reviewer_certainty']


,question,answer,story,round-A_reviewer_output,round-A_reviewer_reasoning,round-A_reviewer_score,round-A_reviewer_certainty
0,Aria lived in a city and had a pet cat named O...,False,"Once upon a time, in a small, enchanted villag...",{'reasoning': 'The statement claims that Aria ...,The statement claims that Aria had a pet cat n...,2,90
1,Aria discovered a hidden waterfall deep in the...,True,"Once upon a time, in a small, enchanted villag...",{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the story explic...,1,95
2,The dragon Aria saved was as large as a horse.,False,"Once upon a time, in a small, enchanted villag...",{'reasoning': 'The statement claims that the d...,The statement claims that the dragon Aria save...,2,100
3,The village of Mistralwood is located near a d...,False,"In the quiet village of Mistralwood, nestled b...",{'reasoning': 'The statement claims that the v...,The statement claims that the village of Mistr...,2,90
4,The lake is called Moonlit Mirror because it r...,True,"In the quiet village of Mistralwood, nestled b...",{'reasoning': 'The statement is true because t...,The statement is true because the story explic...,1,95
5,Elara saw her own reflection and nothing else ...,False,"In the quiet village of Mistralwood, nestled b...",{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because Elara did not s...,2,95
6,Did Luna discover the grove by following a pat...,False,"In the heart of the Enchanted Forest, a tiny f...",{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the context indi...,1,90
7,Were the flowers in the grove capable of speak...,True,"In the heart of the Enchanted Forest, a tiny f...",{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because the flowers in ...,2,90
8,Did Luna solve all three riddles immediately u...,False,"In the heart of the Enchanted Forest, a tiny f...",{'reasoning': 'The statement is FALSE because ...,The statement is FALSE because the story does ...,2,90
9,Elara lived in a bustling city surrounded by s...,False,"In the small village of Windermere, nestled am...",{'reasoning': 'The statement claims that Elara...,The statement claims that Elara lived in a bus...,2,95


In [8]:
reviewer.memory[11]

{'system_prompt': "Your name is: <<reviewer>> Your backstory is: <<A frequent book reader>>. Your task is to review input itmes with the following description: <<TRUE/FALSE questions about stories>>. Your final output should have the following keys: reasoning (<class 'str'>), score (<class 'int'>), certainty (<class 'int'>).",
 'model_args': {'max_tokens': 200, 'temperature': 0.1},
 'input_prompt': "**Review the input item below and complete the scoring task as instructed:** --- **Input item:** <<Review Task ID: A-11 === question === The treasure was believed to be buried beneath the village square.>> **Scoring task:** <<Decide if the input statement is True or False given the provided story in the provided context>> --- **Instructions:** 1. **Score** the input item using only the values in this set: [1, 2]. 2. Follow these rules when determining your score: <<Score 1 if the statement is TRUE and 2 if the statement is FALSE.>>. 3. After assigning a score, report your certainty level as